In [ ]:
import sys
import pandas as pd
import torch

In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset

In [ ]:
from pysentimiento.preprocessing import preprocess_tweet

In [ ]:
pwd

In [11]:
pd.set_option("max_colwidth", 300)

In [269]:
df = pd.read_csv(r'text-extraction/[labeling]dataset-01-sorted.csv')
df

,id_slang,slang,sentence,label,char_count,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,75,anda,Anda,0,4,NaN,NaN,NaN,NaN
1,2139,cagón,cagón,0,5,NaN,NaN,NaN,NaN
2,1392,pesado,pesado,0,6,NaN,NaN,NaN,NaN
3,275,carajo,carajo,0,6,NaN,NaN,NaN,NaN
4,184,cabrón,Cabrón,0,6,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21389,41,achorado,"y los tres chiflados siguen cantando a voz en cuello y en inglés y con toda la concha del mundo, y yo ahí sentadito y timidísimo escuchándolos embobado y mirándole de reojo el pechito de gaviota al loco mariano, y los dos ex convictos de la otra mesa siguen mirándonos feo y yo presiento que esto...",0,3568,NaN,NaN,NaN,NaN
21390,548,concha,"y los tres chiflados siguen cantando a voz en cuello y en inglés y con toda la concha del mundo, y yo ahí sentadito y timidísimo escuchándolos embobado y mirándole de reojo el pechito de gaviota al loco mariano, y los dos ex convictos de la otra mesa siguen mirándonos feo y yo presiento que esto...",0,3568,NaN,NaN,NaN,NaN
21391,1065,macha,"y los tres chiflados siguen cantando a voz en cuello y en inglés y con toda la concha del mundo, y yo ahí sentadito y timidísimo escuchándolos embobado y mirándole de reojo el pechito de gaviota al loco mariano, y los dos ex convictos de la otra mesa siguen mirándonos feo y yo presiento que esto...",0,3568,NaN,NaN,NaN,NaN
21392,2161,cojudo,"y los tres chiflados siguen cantando a voz en cuello y en inglés y con toda la concha del mundo, y yo ahí sentadito y timidísimo escuchándolos embobado y mirándole de reojo el pechito de gaviota al loco mariano, y los dos ex convictos de la otra mesa siguen mirándonos feo y yo presiento que esto...",0,3568,NaN,NaN,NaN,NaN


In [270]:
new_df = df[df.label.isin(["1","2","3"])]
#new_df = new_df.loc[:, ['id_slang','slang', 'sentence', 'label']]
new_df = new_df.loc[:, ['sentence', 'label']]
new_df = new_df.rename(columns={"sentence":"text"})
new_df = new_df.reset_index(drop=True)
new_df = new_df.astype({'label':'int'})
new_df

,text,label
21,Oe Chacal,2
27,Anda vete,3
31,a la firme,2
37,"Chau, cojo",3
42,Tengo plata,2
...,...,...
1955,Ahorita la chola te trae tu desayuno,1
1959,"Siempre he sacado buenas notas, papá",1
1978,Hoy reventamos a esos conchasumadres,3
1979,Un día me cae un piedrón y la cagada,3


In [271]:
new_df.sample(n = 30)

,text,label
110,Te ha fregado,3
856,"¿Qué? Sí, era una conchuda",3
1671,tremenda fiesta se armó en mi depa,1
1505,ojalá que esté muerto ese huevón,3
719,"Pero allá tú, es tu plata",3
265,Gallinas de mierda,3
845,los aliancistas son negros,3
1488,"Cómo quieres te entienda, carajo",3
499,pero llegó la tombería,3
493,Templado hasta el cien,3


In [79]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 412 entries, 21 to 1997
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   slang     412 non-null    object
 1   sentence  412 non-null    object
 2   label     412 non-null    object
dtypes: object(3)
memory usage: 12.9+ KB


trainable model parameters: 108789507
all model parameters: 108789507
percentage of trainable model parameters: 100.00%


In [130]:
##Training

In [131]:
from datasets import load_dataset

ds = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "spanish")

ds

Found cached dataset tweet_sentiment_multilingual (/home/darkstar/.cache/huggingface/datasets/cardiffnlp___tweet_sentiment_multilingual/spanish/0.1.0/936afd3cde120393429606f681b3b48d526873c45114068973f71e296ce80605)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1839
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 324
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 870
    })
})

In [132]:
ds["train"][:10]

{'text': ['estoy hasta el ojete de que me digáis que tengo cara de mala leche',
  '@user Por?  Tenía pensado verla después de la segunda de Daredevil',
  'Esto de estar feliz mola',
  'Ya no es tan divertido',
  '@user te recuerdo que soy una persona que tiene criterio, equivocado, pero lo tengo',
  '@user @user @user con una pequeña donación hará felices a miles de chicas que no tienen  #asociacionmariloli',
  'He probado una nueva espuma para el pelo y sí que lo deja más rizado pero se queda como efecto gomina y no me gusta.',
  '@user Ojalá pudiera darte el abrazo en vivo o al menos acompañarte hoy. Siento mucho lo de tu primo',
  '@user aquí tienes a mi bae aka una egipcia preciosa que esta aprendiendo español. Os llevareis bien  @user',
  '@user ya somos dos, que triste'],
 'label': [0, 1, 2, 0, 1, 2, 0, 1, 2, 0]}

In [133]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "pysentimiento/robertuito-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128


In [134]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 108789507
all model parameters: 108789507
percentage of trainable model parameters: 100.00%


In [135]:
from pysentimiento.preprocessing import preprocess_tweet
preprocessed_ds = ds.map(lambda ex: {"text": preprocess_tweet(ex["text"], lang="es")})

Loading cached processed dataset at /home/darkstar/.cache/huggingface/datasets/cardiffnlp___tweet_sentiment_multilingual/spanish/0.1.0/936afd3cde120393429606f681b3b48d526873c45114068973f71e296ce80605/cache-63d17802209172be.arrow
Loading cached processed dataset at /home/darkstar/.cache/huggingface/datasets/cardiffnlp___tweet_sentiment_multilingual/spanish/0.1.0/936afd3cde120393429606f681b3b48d526873c45114068973f71e296ce80605/cache-383aafcdcf8ebb9e.arrow
Loading cached processed dataset at /home/darkstar/.cache/huggingface/datasets/cardiffnlp___tweet_sentiment_multilingual/spanish/0.1.0/936afd3cde120393429606f681b3b48d526873c45114068973f71e296ce80605/cache-085d67da889331b0.arrow


In [136]:
tokenized_ds = preprocessed_ds.map(
    lambda batch: tokenizer(batch["text"], padding=False, truncation=True),
    batched=True, batch_size=32
)

Loading cached processed dataset at /home/darkstar/.cache/huggingface/datasets/cardiffnlp___tweet_sentiment_multilingual/spanish/0.1.0/936afd3cde120393429606f681b3b48d526873c45114068973f71e296ce80605/cache-378eb711774401ee.arrow
Loading cached processed dataset at /home/darkstar/.cache/huggingface/datasets/cardiffnlp___tweet_sentiment_multilingual/spanish/0.1.0/936afd3cde120393429606f681b3b48d526873c45114068973f71e296ce80605/cache-8f0e3e6aba12a471.arrow
Loading cached processed dataset at /home/darkstar/.cache/huggingface/datasets/cardiffnlp___tweet_sentiment_multilingual/spanish/0.1.0/936afd3cde120393429606f681b3b48d526873c45114068973f71e296ce80605/cache-d8f4cdad7ff5d52f.arrow


In [139]:
import numpy as np
import evaluate

f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")

def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    
    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="macro"))
    return results

In [140]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

training_args = TrainingArguments(
    per_device_train_batch_size=32,
    output_dir="test_trainer",
    do_eval=True,
    evaluation_strategy="epoch",
    num_train_epochs=5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

{'eval_loss': 0.6580564379692078, 'eval_f1': 0.6903269569936237, 'eval_recall': 0.7006172839506174, 'eval_runtime': 0.5642, 'eval_samples_per_second': 574.286, 'eval_steps_per_second': 72.672, 'epoch': 1.0}
{'eval_loss': 0.6773273348808289, 'eval_f1': 0.7105705628814066, 'eval_recall': 0.7098765432098766, 'eval_runtime': 0.5368, 'eval_samples_per_second': 603.63, 'eval_steps_per_second': 76.385, 'epoch': 2.0}
{'eval_loss': 0.8039754033088684, 'eval_f1': 0.7233416789600859, 'eval_recall': 0.7222222222222222, 'eval_runtime': 0.5677, 'eval_samples_per_second': 570.688, 'eval_steps_per_second': 72.217, 'epoch': 3.0}
{'eval_loss': 0.9425756931304932, 'eval_f1': 0.7107606814918214, 'eval_recall': 0.7129629629629629, 'eval_runtime': 0.5865, 'eval_samples_per_second': 552.389, 'eval_steps_per_second': 69.901, 'epoch': 4.0}
{'eval_loss': 1.0149219036102295, 'eval_f1': 0.7035626068624633, 'eval_recall': 0.70679012345679, 'eval_runtime': 0.5677, 'eval_samples_per_second': 570.716, 'eval_steps_per

TrainOutput(global_step=290, training_loss=0.3351433589540679, metrics={'train_runtime': 51.341, 'train_samples_per_second': 179.097, 'train_steps_per_second': 5.649, 'train_loss': 0.3351433589540679, 'epoch': 5.0})

In [141]:
trainer.evaluate(tokenized_ds["test"])

{'eval_loss': 0.9976394176483154, 'eval_f1': 0.7093753726468303, 'eval_recall': 0.7126436781609197, 'eval_runtime': 1.4433, 'eval_samples_per_second': 602.768, 'eval_steps_per_second': 75.519, 'epoch': 5.0}


{'eval_loss': 0.9976394176483154,
 'eval_f1': 0.7093753726468303,
 'eval_recall': 0.7126436781609197,
 'eval_runtime': 1.4433,
 'eval_samples_per_second': 602.768,
 'eval_steps_per_second': 75.519,
 'epoch': 5.0}

In [142]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [273]:
new_df.sample(n = 30)

,text,label
80,habla como faite,2
320,Me la voy a cachar por cojuda»,3
254,Sus viejos son una ladilla,3
292,si todas esas huevadas vienen,2
244,Nos vamos a cagar de risa,1
35,habla Misterio,2
128,Él se sacó la gorra,2
333,me manda plata todos los meses,1
62,Lo puse al palo,2
388,El Viejo estaba sentado en un sofá,2


In [236]:
text = new_df["text"][409]
text

'Hoy reventamos a esos conchasumadres'

In [202]:
#text = "Esta pelicula tuvo una horrible y fea trama"
encoded_input = tokenizer(text, return_tensors='pt').input_ids
encoded_input = encoded_input.to('cuda')

In [203]:
encoded_input

tensor([[    0,   747, 20142,   506,   412,  1623,   461,  2130,  5205,  3205,
             2]], device='cuda:0')

In [204]:
logits = model(encoded_input).logits
logits

tensor([[ 2.8686, -0.7646, -2.2725]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [205]:
print(f'logits [NEGATIVE, NEUTRAL, POSITIVE]: {logits.tolist()[0]}')

logits [NEGATIVE, NEUTRAL, POSITIVE]: [2.868593454360962, -0.7646494507789612, -2.272451162338257]


In [206]:
# Print the probabilities for [not hate, hate]
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [NEGATIVE, NEUTRAL, POSITIVE]: {probabilities}')

probabilities [NEGATIVE, NEUTRAL, POSITIVE]: [0.9687275290489197, 0.025603795424103737, 0.005668580532073975]


In [216]:
from datasets import Dataset

In [242]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1839
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 324
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 870
    })
})

In [275]:
# label encode
def label_encode(x):
    if x == 1:#positive
        return 2
    if x == 2:#neutral
        return 1
    if x == 3:#negative
        return 0

In [276]:
new_df["label"] = new_df["label"].apply(lambda x: label_encode(x))

In [277]:
new_df

,text,label
0,Oe Chacal,1
1,Anda vete,0
2,a la firme,1
3,"Chau, cojo",0
4,Tengo plata,1
...,...,...
407,Ahorita la chola te trae tu desayuno,2
408,"Siempre he sacado buenas notas, papá",2
409,Hoy reventamos a esos conchasumadres,0
410,Un día me cae un piedrón y la cagada,0


In [294]:
#dataset_lang = Dataset.from_pandas(new_df, split="test")
dataset_lang2 = Dataset.from_pandas(new_df)

In [288]:
dataset_lang["label"][:20]

[1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 1, 1, 1, 1, 1, 1, 0]

In [296]:
dataset_lang2

Dataset({
    features: ['text', 'label'],
    num_rows: 412
})

In [298]:
#split train test
from sklearn.model_selection import train_test_split


random_state=1
train_df_, val_df = train_test_split(new_df, 
                                    test_size=0.10, 
                                    random_state=random_state, 
                            stratify=new_df.label.values)

In [299]:

train_df, test_df = train_test_split(train_df_, 
                                    test_size=0.10, 
                                    random_state=42, 
                            stratify=train_df_.label.values)

In [300]:
dataset_lang2 = Dataset.from_pandas(new_df)

,text,label
116,Ese es mi cachorro,2
185,"Movía mi pelota, flaco",2
15,Estoy a punto,1
372,"Mi taita, como les contaba, murió",0
359,Él fue muy ambicioso desde chico,2
...,...,...
303,y les podría caer un dinerito,2
374,Quédate con esa plata y déjame ir,0
146,Enantes subió al sol,1
26,Fui un huevón,0


In [283]:
tokenized_lang_ds = train_df.map(
    lambda batch: tokenizer(batch["text"], padding=True, truncation=True),
    batched=True, batch_size=32
)

Map:   0%|          | 0/412 [00:00<?, ? examples/s]

In [284]:
tokenized_lang_ds

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 412
})

In [285]:
trainer.evaluate(tokenized_lang_ds)

{'eval_loss': 1.1407015323638916, 'eval_f1': 0.5827145245031832, 'eval_recall': 0.608647457434795, 'eval_runtime': 0.3768, 'eval_samples_per_second': 1093.289, 'eval_steps_per_second': 137.988, 'epoch': 5.0}


{'eval_loss': 1.1407015323638916,
 'eval_f1': 0.5827145245031832,
 'eval_recall': 0.608647457434795,
 'eval_runtime': 0.3768,
 'eval_samples_per_second': 1093.289,
 'eval_steps_per_second': 137.988,
 'epoch': 5.0}